In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as dates
import sys
import os
import numpy as np
from re import split, findall
import xarray as xr
from types import SimpleNamespace
sys.path.insert(1, f"{os.path.abspath(os.path.join(os.path.abspath(''), '../'))}")
from src.utils import get_pars_from_ini, make_dir


In [2]:
def moment_nth(sr_nd, dict_diameters, moment):
    mn = sr_nd * dict_diameters['delta_diam'] * dict_diameters['diameters'] ** moment
    return mn.sum()


class Ict2df(object):
    def __init__(self, _file):
        self.path_file = _file
        self.dt, self.dt_sizes, self.sizes, self.header, self.file_type, self.bin_cent, self.dsizes = self._get_meta()
        self.instrument = self.path_file.split('/')[-1].split('-')[-1].split('_')[0]
        self.df = self._read_file()

    def _get_meta(self):
        with open(self.path_file, 'r') as f:
            lines = f.readlines()
            header, file_type = findall(r"\d*\.\d+|\d+", lines[0])
            sizes = np.array([float(''.join(findall(r"\d*\.\d+|\d+", i[i.find('(') + 1: i.find(')')])[:1]))
                              for i in lines[:200] if i.startswith('cbin')])
            dsizes = sizes[1:] - sizes[:-1]
            dsizes = np.append(dsizes, dsizes[-1])
            bin_cent = (sizes[1:] - sizes[:-1]) / 2 + sizes[:-1]
            bin_cent = np.append(bin_cent, sizes[-1] + dsizes[-1])
            dt_sizes = {i: j for i, j in zip(bin_cent, dsizes)}
            try:
                dt = pd.to_datetime(''.join(lines[6].split(',')[:3]), format='%Y%m%d', utc=True)
                return dt, dt_sizes, sizes, int(header) - 1, file_type, bin_cent, dsizes 
            except ValueError:
                dt = pd.to_datetime(''.join(lines[6].split(',')[:3]).replace(' ', ''), format='%Y%m%d', utc=True)
                return dt, dt_sizes, sizes, int(header) - 1, file_type, bin_cent, dsizes 

    def _read_file(self):
        df_fcdp = pd.read_csv(self.path_file, skiprows=self.header, header=0, na_values=[-999, -9.99])
        df_fcdp['time'] = df_fcdp.Time_Start.map(lambda x: self.dt + pd.to_timedelta(x, unit='seconds'))
        df_fcdp['time'] = df_fcdp['time'].map(lambda x: x.to_datetime64())
        df_fcdp.index = df_fcdp['time']
        return df_fcdp

In [4]:
location = split(', |_|-|!', os.popen('hostname').read())[0].replace("\n", "")
path_data = get_pars_from_ini(campaign='loc')[location]['path_data']

# Data
_file = f'{path_data}/data/LAWSON_PAUL/FCDP/CAMP2Ex-FCDP_P3B_20190824_R1.ict'
_file1 = f'{path_data}/data/LAWSON_PAUL/Hawk2DS10/CAMP2Ex-Hawk2DS10_Learjet_20190907_R0_L1.ict'
_file2 = f'{path_data}/data/LAWSON_PAUL/Hawk2DS50/CAMP2Ex-Hawk2DS50_Learjet_20190907_R0_L1.ict'
_file3 = f'{path_data}/data/LAWSON_PAUL/HawkFCDP/CAMP2Ex-HawkFCDP_P3B_20191005_R1.ict'
_file4 = f'{path_data}/data/LAWSON_PAUL/HVPS/CAMP2Ex-HVPS_Learjet_20190907_R0_L1.ICT'

_files = [_file, _file1, _file2, _file3, _file4]

for i in _files:
    print(i)
    _type = i.split('/')[-1].split('-')[-1].split('_')[0]
    files = glob.glob(f'{path_data}/data/LAWSON_PAUL/{_type}/CAMP2Ex-{_type}*')
    example = Ict2df(i)
    ls_pd = [Ict2df(i).df for i in files]
    df_all = pd.concat(ls_pd)
    df_all.to_pickle(f'{path_data}/data/LAWSON_PAUL/{_type}/{_type}_all.pkl')

/data/keeling/a/alfonso8/gpm/camp2ex/data/LAWSON_PAUL/FCDP/CAMP2Ex-FCDP_P3B_20190824_R1.ict
/data/keeling/a/alfonso8/gpm/camp2ex/data/LAWSON_PAUL/Hawk2DS10/CAMP2Ex-Hawk2DS10_Learjet_20190907_R0_L1.ict
/data/keeling/a/alfonso8/gpm/camp2ex/data/LAWSON_PAUL/Hawk2DS50/CAMP2Ex-Hawk2DS50_Learjet_20190907_R0_L1.ict
/data/keeling/a/alfonso8/gpm/camp2ex/data/LAWSON_PAUL/HawkFCDP/CAMP2Ex-HawkFCDP_P3B_20191005_R1.ict
/data/keeling/a/alfonso8/gpm/camp2ex/data/LAWSON_PAUL/HVPS/CAMP2Ex-HVPS_Learjet_20190907_R0_L1.ICT


In [5]:
df_all.attrs

{}

In [ ]:
inicio = pd.to_datetime('2019-09-07 00:00:00', format='%Y-%m-%d %H:%M:%S')
fin = pd.to_datetime('2019-09-07 00:05:00', format='%Y-%m-%d %H:%M:%S') 
lis = pd.date_range(inicio, fin, freq='1min')

In [ ]:
filtered = df_all[(df_all.index > inicio) & (df_all.index < fin)].filter(like='cbin')

In [ ]:
example.bin_cent

In [ ]:
cdict = {'red': ((0., 1, 1),
                 (0.05, 1, 1),
                 (0.11, 0, 0),
                 (0.66, 1, 1),
                 (0.89, 1, 1),
                 (1, 0.5, 0.5)),
         'green': ((0., 1, 1),
                   (0.05, 1, 1),
                   (0.11, 0, 0),
                   (0.375, 1, 1),
                   (0.64, 1, 1),
                   (0.91, 0, 0),
                   (1, 0, 0)),
         'blue': ((0., 1, 1),
                  (0.05, 1, 1),
                  (0.11, 1, 1),
                  (0.34, 1, 1),
                  (0.65, 0, 0),
                  (1, 0, 0))}

my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)

In [ ]:
plt.figure(figsize=(15,4.5))
# plt.rc(group='text', fontsize=14, usetex=True)
plt.pcolormesh(filtered.index.values, example.bin_cent, filtered.T, cmap=my_cmap,  vmin=0, vmax=10000, shading='auto') #  vmin=0, vmax=5,
# plt.pcolormesh(filtered.columns,  filtered.index.values, filtered, cmap=my_cmap, shading='auto') #  vmin=0, vmax=5,

plt.colorbar(pad=0.01, aspect=20).set_ticks(np.arange(0,10000,1000))
plt.ylabel(r'$Di\acute{a}metro, mm$', fontsize='x-large')
# plt.yticks(np.arange(0,9,2))
plt.xlabel('$Hora, HLC$', fontsize='x-large')
plt.title('$N(D), \log_{10} (\# \ m^{-3} mm^{-1}) $',  position=(0.8, 0.85), fontsize='x-large')
plt.gca().xaxis.set_major_formatter(dates.DateFormatter('%H:%M')) 

In [ ]:
# ds = xr.Dataset.from_dataframe(df_all)
# ds.attrs['dsizes'] = example.dsizes
# ds.attrs['ls_sizes'] = example.sizes
# ds.attrs['bin_cent'] = example.bin_cent

In [ ]:
# folder = f'{path_data}/data/LAWSON_PAUL/{_type}/zarr'
# make_dir(folder)
# ds.to_zarr(folder)

In [ ]:
#
# Plotting PDS
_date, dt_sizes = example.dt, example.dt_sizes
values = df_all[df_all.conc > 100000].filter(like='cbin').iloc[0]
fig, ax = plt.subplots(1, 1)
ax.scatter(x=list(dt_sizes.keys()), y=values, c='k', marker='*')
ax.set_yscale('log')
ax.set_ylabel('Number Concentration (#/L)')
ax.set_xlabel('Diameter (um)')
ax.set_xlim(0, 150)
ax.grid()
plt.show()
print(1)
#



In [ ]:
df_all[df_all.conc > 1000].filter(like='cbin')

In [ ]:
df_all[df_all.conc > 10000]